
Programmieren 3 - Optimierung

Peter Rösch, Fakultät für Informatik

Hochschule Augsburg, 2023/2024

# Optimierung - Einführung

Literatur:
   
* Michal Jaworski, Tarek Ziadé: "Expert Python Programming", PACKT Publishing, [eBook](https://learning.oreilly.com/library/view/expert-python-programming/9781801071109/Text/Chapter_13.xhtml#_idParaDest-260), Kap. 13
* [Wikipedia](http://en.wikipedia.org/wiki/Program_optimization)

Frage: Warum sagt Donald E. Knuth "[Premature Optimisation](https://www.goodreads.com/quotes/1194913-premature-optimization-is-the-root-of-all-evil) is the root of all evil"

Donald Knuth: Structured Programming with go to Statments. Computing Surveys **4** (1974) 261-301, [pdf, S. 39 rechts unten](http://web.archive.org/web/20130731202547/http://pplab.snu.ac.kr/courses/adv_pl05/papers/p261-knuth.pdf)

## Bestätigt aus eigener Erfahrung ...

* Kent Beck: "Make it Work, make it right, make it fast" [Programmer Friday](https://tknilsson.com/2018/05/25/programmer-friday-make-it-work-make-it-right-make-it-fast/)
* Zen of Python: "Readability counts'
* Die Optimierung von Algorithmen (Laufzeitkomplexität) bringt mehr als die Optimierung einer Implementierung.
* Bevor man optimiert, sollte man die Anwendung klassifizieren (I/O- oder rechenlastig, Programm im Dauereinsatz oder "Einweg-Skript" etc).
* Optimierung "aus dem Bauch heraus" bringt wenig. Besser ist der systematische Ansatz mit Messungen (Profiling).
* Werkzeuge zur Optimierung werden immer stärker automatisiert, Augen offen halten ... 

## Beispiel: TSP

Eine Implementierung, die deutlich schneller als das einfache Ausprobieren aller Permutationen ist, finden Sie im Notebook *TSP_dynamisch.ipynb*. Hier ist der Schlüssel die [dynamische Programmierung](https://de.wikipedia.org/wiki/Dynamische_Programmierung).

## Vereinfachung der dynamischen Programmierung mit *lru_cache*

Quelle: Dawud Ibrahim Ismail: [Memoization in Python](https://idawud.tech/memoization-in-python).

Idee: Verwendung des *lru-cache*

In [ ]:
def fibonacci(n: int) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [ ]:
import timeit

timeit.timeit(
    "fibonacci(35)", setup="from __main__ import fibonacci", number=5
)

In [ ]:
import functools


@functools.lru_cache(maxsize=128)
def fibonacci_dynamisch(n: int) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci_dynamisch(n - 1) + fibonacci_dynamisch(n - 2)

In [ ]:
timeit.timeit(
    "fibonacci_dynamisch(35)",
    setup="from __main__ import fibonacci_dynamisch",
    number=5,
)

In [ ]:
fibonacci_dynamisch.cache_clear()

In [ ]:
# Vertrauensbildende Massnahme
fibonacci(35) - fibonacci_dynamisch(35)

**Frage:** Wann ist bei der Verwendung von *lru_cache* Vorsicht geboten?

## Optimierung I/O-lastiger Anwendungen

* Zunächst seriell zum Laufen bringen (Referenz) ...
* Profiling
* Nebenläufigkeit mit [*asyncio*](https://docs.python.org/3.10/library/asyncio.html).
* Intelligente Caching-Strategie.
* Vorhersehbare I/O-Operationen vorziehen.

## Optimierung rechenintensiver Anwendungen

* Zunächst mit dem Fokus auf Lesbarkeit und Strukturierung in kleine Einheiten zum Laufen bringen (Referenz).
* Profiling
* Optimierung (Speicherzugriffe, Datentypen, Möglichkeiten der verwendeten CPU/GPU)
* Parallelisierung (Nutzung aller Kerne/Threads)
* Verteilung (Cluster)

# Systematische Optimierung - Werkzeuge

Bei der Optimierung sollte man unbeding systematisch vorgehen und sich nicht auf das "Bauchgefühl" verlassen.

Im Buch "Programming in Python 3" von Mark Summerfield ([eBook](https://learning.oreilly.com/library/view/programming-in-python/9780321699909)) finden Sie einen interessanten Abschnitt zum Thema Profiling (Kap. 9).

Folgendes Programm berechnet den Mittelwert aus vielen Zufallszahlen, auf die die *sin*-Funktion angewendet wurde:

In [ ]:
import random, math


def zufalls_mittelwert_python(seed: int, n: int) -> float:
    random.seed(seed)
    sum = 0.0
    for i in range(n):
        sum += math.sin(random.random())
    return sum / n

Mit *timeit* kann die absolute Ausführungszeit bestimmt werden:

In [ ]:
%timeit zufalls_mittelwert_python(17635, 10**6)

Flaschenhälse ('Bottlenecks') oder 'hot spots' können mit einem profiler berechnet werden. Genaueres finden Sie in der [Dokumentation](https://docs.python.org/3.10/library/profile.html). IPython bietet das magische Kommando *%prun*, siehe 

In [ ]:
%prun zufalls_mittelwert_python(17635, 10**5)

Wesentlich komfortabler und mächtiger ist das Programm [scalene](https://pypi.org/project/scalene).


In [ ]:
%load_ext scalene

In [ ]:
import TSP

In [ ]:
%%scalene --reduced-profile --cpu-only
result = TSP.shortest_closed_path(TSP.staedte_positionen[0:0], TSP.staedte_positionen[1:10])

Aufruf im Terminal:

    scalene --html --outfile scalene_TSP.html TSP.py
    firefox scalene_TSP.html

# Das Paket numpy

Fragen:
 1. Warum sind Python-Listen für effiziente numerische Berechnungen nicht gut geeignet?
 1. Welche Eigenschaften sollte eine Datenstruktur für effiziente numerische Berechnungen besitzen?
 1. Wie sollte eine Integration effizienter Numerik in Python aussehen?

## numpy - Grundlagen

Quellen:
  - [numpy home page](http://www.numpy.org)
  - [numpy-Dokumentation](http://docs.scipy.org/doc)
  - [numpy-Buch (pdf)](http://csc.ucdavis.edu/~chaos/courses/nlp/Software/NumPyBook.pdf)
  
numpy nutzt die [intel math kernel library](https://software.intel.com/content/www/us/en/develop/tools/math-kernel-library.html) zur effizienten Nutzung der CPU.
    

In [ ]:
import math

In [ ]:
# Anzahl der Berechnungen
N = 10000

In [ ]:
%%timeit
l = [ math.sin(math.pi * i / N) for i in range(N) ]

Mit numpy-Arrays können Berechnungen wesentlich schneller durchgeführt werden:

In [ ]:
import numpy as np

In [ ]:
%%timeit
y = np.sin( np.arange(0, math.pi, math.pi / N) )

Im Gegensatz zu Python-Listen sind numpy-Arrays statisch, d.h. ihre Größe ändert sich nicht, und können nur Elemente des gleichen Typs (z.B. numpy.float32 oder numpy.float64) enthalten.

In [ ]:
a = np.arange(0, 10, 0.01, dtype=np.float32)
print("shape:", a.shape)
print("type: ", type(a))
print("dtype: ", a.dtype)

Numpy-Funktionen wirken *elementweise* auf numpy-Arrays.

In [ ]:
x = np.arange(0, math.pi, math.pi / 4)
y = np.sin(x) ** 2
print("sin(x)**2:", y)
print("Mittelwert von y:", np.mean(y))

NumPy kann mit n-Dimensionalen Feldvariablen rechnen. Häufig verwendet werden Matrizen (dim=2). Bitte beachten Sie: 

Der Operator '\*' bezeichnet die *elementweise* Multiplikation. Die Matrizen-Multiplikation heißt *numpy.dot* oder @ in neueren Python-Versionen!

In [ ]:
matrix1 = np.random.normal(2, 0.1, size=(3, 5))
matrix2 = np.random.normal(3, 0.2, size=(5, 2))
print("matrix1:", matrix1)
print("numpy.dot(matrix1, matrix2):\n", np.dot(matrix1, matrix2))
# Vorsicht, Reihenfolge! Die naechste Zeile gibt einen Fehler
# print(np.dot(matrix2, matrix1))
# Wenn man die Reihenfolge ändern will, muss man transponieren:
print("np.dot(matrix2.T, matrix1.T):\n", np.dot(matrix2.T, matrix1.T))

Matrizen können z.B. auch invertiert oder zerlegt werden. Die entsprechenden Funktionen finden sich im Paket *numpy.linalg*.

In [ ]:
# Nur quadratische Matzizen können invertiert werden ...
m1 = np.random.normal(1, 0.2, size=(4, 4)).astype(np.float64)
m1Inv = np.linalg.inv(m1)
print("np.dot(m1, m1Inv):\n", np.dot(m1, m1Inv))

In [ ]:
# Singulaerwertzerlegung
np.linalg.svd(m1[0:3, 0:3])

Die numpy-Syntax ist oft intuitiv fassbar:

In [ ]:
a = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
print("a:\n", a)
b = a.copy()
b[b % 2 == 0] = 77
print("b:\n", b)
c = np.where(a > 5, a / 2, a)
print("c:", c)

Weitere Beispiele finden Sie im oben schon erwähnten Buch oder auf der [Homepage](http://www.numpy.org/).

## Optimierung - numexpr

Das Paket [numexpr](https://github.com/pydata/numexpr) übersetzt *numpy*-Ausdrücke, die als Zeichenkette übergeben werden, in opcodes, die in einer virtuellen Maschine **parallelisiert** ausgeführt werden.

In [ ]:
import numpy as np

x = np.random.uniform(low=0, high=10, size=10**6)

In [ ]:
def test_function_numpy(a):
    a = a + 3
    b = np.sin(a) * a**2.3 + 3 * a
    return b

In [ ]:
%timeit test_function_numpy(x)

In [ ]:
import numexpr


def test_function_numexpr(a):
    a = numexpr.evaluate("a + 3")
    b = numexpr.evaluate("sin(a) * a ** 2.3 + 3 * a")
    return b

In [ ]:
%timeit test_function_numexpr(x)

# Optimierung - numba

Bei [*numba*](http://numba.pydata.org/) handelt es sich um einen 'Just-in-time-Compiler', der auf  [*llvm*](http://llvm.org) basiert. Numba bietet eine sehr gute Optimierung mit extrem wenig Aufwand.

In [ ]:
from numba import jit

import random, math


@jit
def zufalls_mittelwert_numba(seed, n):
    random.seed(seed)
    sum = 0.0
    for i in range(n):
        sum += math.sin(random.random())
    return sum / n

In [ ]:
#zur Erinnerung:
%timeit zufalls_mittelwert_python(17635, 10**6)

In [ ]:
%timeit zufalls_mittelwert_numba(17635, 10**6)

## Parallelisierung mit numba

In [ ]:
from numba import jit, prange

import random, math


@jit(parallel=True, nopython=True)
def zufalls_mittelwert_numba_parallel(seed, n):
    random.seed(seed)
    sum = 0.0
    for i in prange(n):
        sum += math.sin(random.random())
    return sum / n

In [ ]:
%timeit zufalls_mittelwert_numba(17635, 10**6)

In [ ]:
%timeit zufalls_mittelwert_numba_parallel(17635, 10**6)

numba kann Funktionen, die mit numpy-Arrays arbeiten, automatisch parallelisieren und auch die SIMD-Instruktionen der CPU verwenden:

In [ ]:
import numpy as np
from numba import jit

x = np.random.uniform(low=0, high=10, size=10**6)


@jit(parallel=True, nopython=True)
def test_function_numba(a):
    a = a + 3
    b = np.sin(a) * a**2.3 + 3 * a
    return b

In [ ]:
%timeit test_function_numba(x) 

In [ ]:
%timeit test_function_numpy(x)

In [ ]:
%timeit test_function_numexpr(x)

# Ausblick: GPU-Computing

Quelle: [numba-Dokumentation](https://numba.pydata.org/numba-doc/latest/cuda/random.html).

In [ ]:
from numba import cuda
from numba.cuda.random import (
    create_xoroshiro128p_states,
    xoroshiro128p_uniform_float32,
)
import numpy as np

In [ ]:
# Source: https://numba.pydata.org/numba-doc/latest/cuda/random.html
@cuda.jit
def compute_pi(rng_states, iterations, out):
    thread_id = cuda.grid(1)

    # Compute pi by drawing random (x, y) points and finding what
    # fraction lie inside a unit circle
    inside = 0
    for i in range(iterations):
        x = xoroshiro128p_uniform_float32(rng_states, thread_id)
        y = xoroshiro128p_uniform_float32(rng_states, thread_id)
        if x**2 + y**2 <= 1.0:
            inside += 1

    out[thread_id] = 4.0 * inside / iterations

In [ ]:
def compute_pi_numba_cuda(nr_of_iterations, threads_per_block=64, blocks=64):
    iterations_per_thread = nr_of_iterations // threads_per_block // blocks
    rng_states = create_xoroshiro128p_states(
        threads_per_block * blocks, seed=1
    )
    out = np.zeros(threads_per_block * blocks, dtype=np.float32)
    compute_pi[blocks, threads_per_block](
        rng_states, iterations_per_thread, out
    )
    return out.mean()

In [ ]:
# ohne GPU
# N = 10**5
# mit GPU
N = 10**7
print(compute_pi_numba_cuda(N))

sm_count = 48

for threads_per_block in (4, 8, 16, 32, 64, 128):
    print(f'\n{threads_per_block = }')
    %timeit compute_pi_numba_cuda(N, threads_per_block, 2*sm_count)

In [ ]:
%timeit compute_pi_numba_cuda(N, 1, 1)

Ausgabe, NVIDIA A4000

1.03 s ± 1.91 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Aufgaben, Abgabe 12.12. und 14.12.2023

## Middleware / Docker

1. Nennen und beschreiben Sie zwei wichtige Vorteile, die Entwickler durch den Einsatz einer Middleware haben? (ca. vier Sätze)
1. Vergegenwärtigen Sie sich nochmals die Struktur und Funktionsweise der im Rahmen der Vorlesung vorgestellten verteilten Monte-Carlo-Simulation, die [Ice](http://www.zeroc.com) verwendet.
1. Sie sollen eine verteilte Anwendung zur Berechnung der Werte der allgemeinen harmonischen und der allgemeinen alternierenden harmonischen Reihe erstellen. Eingabegrößen sind: Art der Reihe, $\alpha$ sowie die Grenze $N$, bis zu der die Summation durchgeführt wird. Dabei soll der Server, der die Berechnungen durchführt, in Java und der Client in Python implementiert werden. Als Middleware ist [Ice](http://www.zeroc.com) zu verwenden. Gehen Sie nach dem in der ersten Veranstaltung eingeführten Schema vor, wobei Sie aufgrund der Vorgaben einige Schritte überspringen können. 
1. Erstellen und testen Sie ein Docker-Image für den in den letzten Teilaufgabe erstellten Server. Verwenden Sie dazu eigene Rechner mit der VM *bookworm*.

## $\pi$ nach Monte Carlo: Optimierung mit *numba*

1. Analysieren Sie Ihre Lösung zur Berechnung von $\pi$ quantitativ bezüglich des Ressourcen-Verbrauchs.
1. Welche Schlußfolgerungen zur Vorbereitung der Optimierung können Sie aus den Ergebnissen ziehen?
1. Verwenden Sie *numba*, um eine optimierte Version der $\pi$-Berechnung zu erstellen, vergleichen Sie die Rechenzeit mit der Ihrer parallelisierten Python-Implementierung und erklären Sie Ihre Beobachtung.
1. Erklären Sie die beobachtete Abhängigkeit zwischen dem Parameter *threads_per_block* und der Rechenzeit im oben gegebenen Beispiel zum GPU-Computing auf der NVIDIA RTX A4000.

# Überprüfung

1. Was ist der unterschied zwischen *statistischem* und *deterministischem* Profiling?
1. Nennen Sie zwei häufige Fehler, die im Zusammenhang mit Optimierung gemacht werden. (max. vier Sätze)